 # Multi Query Retrievers
 This tutorial demonstrates how to build and use multi-query retrievers with LangChain.
 We'll start by loading documents, splitting them into chunks, and using embeddings
 to enable similarity search and retrieval.

In [73]:
from langchain.storage import InMemoryByteStore
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from rich import print
import uuid

 ## Step 1: Load Documents
 Here, we load the documents using `TextLoader`. You can replace the file paths
 with your own document paths.

In [74]:
loaders = [
    TextLoader("data/langchain.md"),
    TextLoader("data/langchain2.md"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())
print(f"Loaded {len(docs)} documents")
full_docs = docs
print(full_docs)


Loaded 2 documents

[
    Document(
        metadata={'source': 'data/langchain.md'},
        page_content='Title: Chat models | 🦜️🔗 LangChain\n\nURL Source: 
https://python.langchain.com/docs/concepts/chat_models/\n\nMarkdown 
Content:\nOverview[\u200b](https://python.langchain.com/docs/concepts/chat_models/#overview "Direct link to 
Overview")\n-------------------------------------------------------------------------------------------------------
\n\nLarge Language Models (LLMs) are advanced machine learning models that excel in a wide range of 
language-related tasks such as text generation, translation, summarization, question answering, and more, without 
needing task-specific fine tuning for every scenario.\n\nModern LLMs are typically accessed through a chat model 
interface that takes a list of [messages](https://python.langchain.com/docs/concepts/messages/) as input and 
returns a [message](https://python.langchain.com/docs/concepts/messages/) as output.\n\nThe newest generation of 
chat models offer additional capabilities:\n\n*   [Tool 
calling](https://python.langchain.com/docs/concepts/tool_calling/): Many popular chat models offer a native [tool 
calling](https://python.langchain.com/docs/concepts/tool_calling/) API. This API allows developers to build rich 
applications that enable LLMs to interact with external services, APIs, and databases. Tool calling can also be 
used to extract structured information from unstructured data and perform various other tasks.\n*   [Structured 
output](https://python.langchain.com/docs/concepts/structured_outputs/): A technique to make a chat model respond 
in a structured format, such as JSON that matches a given schema.\n*   
[Multimodality](https://python.langchain.com/docs/concepts/multimodality/): The ability to work with data other 
than text; for example, images, audio, and 
video.\n\nFeatures[\u200b](https://python.langchain.com/docs/concepts/chat_models/#features "Direct link to 
Features")\n-------------------------------------------------------------------------------------------------------
\n\nLangChain provides a consistent interface for working with chat models from different providers while offering 
additional features for monitoring, debugging, and optimizing the performance of applications that use LLMs.\n\n*  
Integrations with many chat model providers (e.g., Anthropic, OpenAI, Ollama, Microsoft Azure, Google Vertex, 
Amazon Bedrock, Hugging Face, Cohere, Groq). Please see [chat model 
integrations](https://python.langchain.com/docs/integrations/chat/) for an up-to-date list of supported models.\n* 
Use either LangChain\'s [messages](https://python.langchain.com/docs/concepts/messages/) format or OpenAI 
format.\n*   Standard [tool calling API](https://python.langchain.com/docs/concepts/tool_calling/): standard 
interface for binding tools to models, accessing tool call requests made by models, and sending tool results back 
to the model.\n*   Standard API for [structuring 
outputs](https://python.langchain.com/docs/concepts/structured_outputs/#structured-output-method) via the 
`with_structured_output` method.\n*   Provides support for [async 
programming](https://python.langchain.com/docs/concepts/async/), [efficient 
batching](https://python.langchain.com/docs/concepts/runnables/#optimized-parallel-execution-batch), [a rich 
streaming API](https://python.langchain.com/docs/concepts/streaming/).\n*   Integration with 
[LangSmith](https://docs.smith.langchain.com/) for monitoring and debugging production-grade applications based on 
LLMs.\n*   Additional features like standardized [token 
usage](https://python.langchain.com/docs/concepts/messages/#aimessage), [rate 
limiting](https://python.langchain.com/docs/concepts/chat_models/#rate-limiting), 
[caching](https://python.langchain.com/docs/concepts/chat_models/#caching) and 
more.\n\nIntegrations[\u200b](https://python.langchain.com/docs/concepts/chat_models/#integrations "Direct link to 
Integrations")\n--------------------

 ## Step 2: Split Documents into Large Chunks
 Using `RecursiveCharacterTextSplitter`, we split documents into smaller chunks
 for more efficient processing and retrieval.

In [75]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
docs = text_splitter.split_documents(docs)



 ## Step 3: Initialize the Vectorstore
 The vectorstore indexes the document chunks for similarity search. We use the `Chroma`
 library with embeddings from Ollama.

In [76]:
vectorstore = Chroma(
    collection_name="full_documents",
    embedding_function=OllamaEmbeddings(model='snowflake-arctic-embed:33m')
)

 ## Step 4: Create Smaller Chunks
 For fine-grained retrieval, we split the documents further into smaller chunks
 and associate metadata for linking with original documents.

In [77]:
from langchain.retrievers.multi_vector import MultiVectorRetriever

# Initialize storage for parent documents
store = InMemoryByteStore()
id_key = "doc_id"

# Create the retriever
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)

doc_ids = [str(uuid.uuid4()) for _ in docs]

# Split into smaller chunks
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
sub_docs = []
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    _sub_docs = child_text_splitter.split_documents([doc])
    for _doc in _sub_docs:
        _doc.metadata[id_key] = _id
    sub_docs.extend(_sub_docs)



 ## Step 5: Add Documents to Vectorstore
 Add the smaller chunks and their metadata to the vectorstore for similarity search.

In [78]:
retriever.vectorstore.add_documents(sub_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))
print(retriever)

MultiVectorRetriever(
    vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x310473310>,
    byte_store=<langchain_core.stores.InMemoryByteStore object at 0x31045e910>,
    docstore=<langchain.storage.encoder_backed.EncoderBackedStore object at 0x31045d010>,
    search_kwargs={}
)



 ## Step 6: Perform a Similarity Search
 Use the retriever to find documents similar to the query "LangChain".

In [79]:
print(retriever.vectorstore.similarity_search("LangChain")[0:2])

[
    Document(
        id='97524e7b-902d-4069-a3ce-06f873e0491f',
        metadata={'doc_id': 'a5154e19-f37d-4514-aaba-68933f778c71', 'source': 'data/langchain.md'},
        page_content='LangChain provides a consistent interface for working with chat models from different 
providers while offering additional features for monitoring, debugging, and optimizing the performance of 
applications that use LLMs.'
    ),
    Document(
        id='fc6cac54-4b72-40f9-a65b-262a30072a65',
        metadata={'doc_id': '7a172ae0-0341-4830-9524-9bb45208da8f', 'source': 'data/langchain.md'},
        page_content='LangChain provides a consistent interface for working with chat models from different 
providers while offering additional features for monitoring, debugging, and optimizing the performance of 
applications that use LLMs.'
    )
]



 ## Step 7: Multi-Modal Retrieval
 Modify the search type to use Maximal Marginal Relevance (MMR) for diverse results.

In [80]:
from langchain.retrievers.multi_vector import SearchType

retriever.search_type = SearchType.mmr
print(retriever.invoke("LangChain")[0:2])

[]



 ## Step 8: Summarize Documents
 Associate summaries with documents using a language model.

In [81]:
import getpass
import os
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama

# Initialize LLM
llm = ChatOllama(model='llama3.2:1b')

from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

chain = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template("Summarize the following document:\n\n{doc}")
    | llm
    | StrOutputParser()
)

# Summarize documents in parallel
summaries = chain.batch(docs, {"max_concurrency": 5})
print(summaries[0:2])

[
    "The document discusses LangChain, a platform for developing large language models (LLMs) that can be used as a
chat model interface. Here's a summary of the key points:\n\n**Overview**\n\n* Large Language Models excel in tasks
like text generation, translation, summarization, and question answering without fine-tuning.\n* Modern LLMs are 
accessed through a chat model interface, taking input messages and returning output 
messages.\n\n**Capabilities**\n\n* **Tool calling**: Many popular chat models have native API for tool calling, 
allowing developers to interact with external services, APIs, and databases.\n* **Structured outputs**: A technique
to return responses in structured formats like JSON that match a given schema.\n* **Multimodality**: The ability to
work with data other than text, such as images, audio, and video.\n\nThese features enable the development of rich 
applications that can interact with external services and perform various tasks.",
    'The document provides an overview of LangChain, a framework for working with chat models from various 
providers. It highlights the following features:\n\n- Integration with multiple chat model providers (e.g., 
Anthropic, OpenAI, Ollama) and their respective formats.\n- Support for messages in either the LangChain or OpenAI 
format.\n- A standard tool calling API to bind tools to models and access their requests and results.\n- Standard 
API for structuring outputs using the `with_structured_output` method.\n- Efficient batching capabilities.\n- A 
rich streaming API.\n\nAdditionally, the document mentions integration with LangSmith for monitoring and debugging 
production-grade applications based on LLMs. It also covers various features such as token usage, rate limiting, 
caching, and more.'
]



 ## Step 9: Add Summaries to Vectorstore
 Store the summaries in the vectorstore for enhanced retrieval capabilities.

In [82]:
vectorstore = Chroma(collection_name="summaries", embedding_function=OpenAIEmbeddings())
store = InMemoryByteStore()
id_key = "doc_id"

retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)

summary_docs = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(summaries)
]

retriever.vectorstore.add_documents(summary_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))



 ## Step 10: Retrieve Summaries
 Perform a similarity search on the summaries to get the most relevant results.

In [83]:
sub_docs = retriever.vectorstore.similarity_search("LangChain")
print("Sub Docs:\n",sub_docs[0])

retrieved_docs = retriever.invoke("LangChain")
print("Retrieved Docs:\n",retrieved_docs[0:2])
# len(retrieved_docs[0].page_content)

Sub Docs:

Document(
    id='810f2212-0166-4293-a9d3-ddf1834072d8',
    metadata={'doc_id': 'a5154e19-f37d-4514-aaba-68933f778c71'},
    page_content="The document describes LangChain's features as follows:\n\n- Provides a consistent interface for 
interacting with various chat models.\n- Offers additional features such as monitoring, debugging, and optimization
for improving application performance with Large Language Models (LLMs)."
)

Retrieved Docs:

[]

In [84]:
# loop through the retrieved documents and print the content of each
for doc in retrieved_docs:
	print(doc.page_content)